In [ ]:
%matplotlib inline

import ROOT, sys, os
from ROOT import std

from larcv import larcv
from larlite import larlite as ll
from larlite import larutil as lu

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.patches as patches

from ROOT import geo2d,cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

import root_numpy as rn
import pandas as pd

geoh=lu.GeometryHelper.GetME()
geo=lu.Geometry.GetME()
larp=lu.LArProperties.GetME()
pygeo = geo2d.PyDraw()

In [ ]:
reco_df = pd.DataFrame(rn.root2rec("../reco_ana.root",treename='LArbysImageTree'))
mc_df   = pd.DataFrame(rn.root2rec("../mc_ana.root",treename='mctree'))

In [ ]:
#Calculating the signal denomenator...
pdg2mass={}
pdg2mass[13]  = 105
pdg2mass[2212]= 938
pdg2mass[2112]= 939.5
pdg2mass[11]  = 0#0.5
pdg2mass[311] = 497.65
def mass(pdg):
    return pdg2mass[pdg]
mass_vec=np.vectorize(mass)

mc_wdf=mc_df.copy()
print mc_wdf.index.size
mc_wdf=mc_wdf.query('energyInit > 100 & energyInit < 500')
print mc_wdf.index.size
mc_wdf=mc_wdf[mc_wdf.daughterPdg_v.apply(lambda x : x.size>=2)]
print mc_wdf.index.size

def containment(row):
    pdg_v   = row['daughterPdg_v']
    einit_v = row['daughterEnergyInit_v']
    edep_v  = row['daughterEnergyDep_v']
    
    eisub_v=einit_v-mass_vec(pdg_v)
  
    init_sum = eisub_v.sum()
    dep_sum  = edep_v.sum()
    
    proton_v=eisub_v[np.where(pdg_v==2212)[0]]
    
    if(np.where(proton_v>80)[0].size>0): pass
    else: return False;
        
    if dep_sum / init_sum < 0.9: return False;

    return True;
    
mc_wdf=mc_wdf[mc_wdf.apply(containment,axis=1)]
mc_wdf_idx = mc_wdf.index

In [ ]:
#Calculating the efficiency numerator
#reco_df_copy will have out analysis cuts built in
reco_wdf = reco_df.ix[mc_wdf_idx].copy()

cuts_v=[]
cuts_v.append('n_vtx3d>0')

for cut in cuts_v:
    reco_wdf = reco_wdf.query(cut)

print reco_wdf.index.size

#filter vertices from, only care about the events that have
#vertex estimated from 3 planes
reco_wdf = reco_wdf[reco_wdf['num_planes_v'].apply(lambda x : np.where(x==3)[0].size)>0]

print reco_wdf.index.size

#cut on the pixel frac
#pxfrac=0.9
#def pixels(x):
#    y=np.hstack(x)
#    return np.where(y>pxfrac)[0].size

#reco_wdf=reco_wdf[reco_wdf['num_pixel_frac_vv'].apply(pixels) > 0]

In [ ]:
#Lets look @ some resolution
